# Camera-Trap-Data-Pipeline

Scripts to process camera trap images.

In [ ]:
# Load important packages
import os
import qgrid
import pandas as pd

## Parameters 

Here we select / define the paramters of the current run: Choose the appropriate cell and execute it.

In [ ]:
###################################
# Grumeti
####################################

SITE='GRU'

SEASON='GRU_S1'

PROJECT_ID='5115'

WORKFLOW_ID='4979'

WORKFLOW_VERSION_MIN='275'

ATTRIBUTION="'University of Minnesota Lion Center + Snapshot Safari + Singita Grumeti + Tanzania'"

LICENSE="'Snapshot Safari + Singita Grumeti'"

In [ ]:
###################################
# RUA
####################################

SITE='RUA'

SEASON='RUA_S1'

PROJECT_ID='5155'

WORKFLOW_ID='4889'

WORKFLOW_VERSION_MIN='797'

ATTRIBUTION=''

LICENSE=''

In [ ]:
###################################
# Mountain Zebra
####################################

SITE='MTZ'

SEASON='MTZ_S1'

PROJECT_ID='5124'

WORKFLOW_ID='8814'

WORKFLOW_VERSION_MIN='247'

ATTRIBUTION=''

LICENSE=''

In [ ]:
###################################
# Karoo
####################################

SITE='KAR'

SEASON='KAR_S1'

PROJECT_ID='7679'

WORKFLOW_ID='8789'

WORKFLOW_VERSION_MIN='237.7'

ATTRIBUTION=''

LICENSE=''

In [ ]:
###################################
# Karoo TEST
####################################

SITE='KAR_TEST'

SEASON='KAR_S1'

PROJECT_ID='7679'

WORKFLOW_ID='8789'

WORKFLOW_VERSION_MIN='237.7'

ATTRIBUTION=''

LICENSE=''

### Set the Paramters

Here we actually set the parameters.

In [ ]:
os.environ["SITE"] = SITE
os.environ["SEASON"] = SEASON
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["WORKFLOW_ID"] = WORKFLOW_ID
os.environ["WORKFLOW_VERSION_MIN"] = WORKFLOW_VERSION_MIN
os.environ["ATTRIBUTION"] = ATTRIBUTION
os.environ["LICENSE"] = LICENSE

Lets check if the parameters have been set correctly.

In [ ]:
print("Selected: site: {} season: {} project_id: {} workflow_id: {} workflow_version_min: {} attribution: {} license: {}".format(
    SITE, SEASON, PROJECT_ID, WORKFLOW_ID, WORKFLOW_VERSION_MIN, ATTRIBUTION, LICENSE))
print(os.environ['SITE'])
print(os.environ['SEASON'])

## Start the Pre-Processing

In [ ]:
# Check Input Structure
!python3 -m pre_processing.check_input_structure \
--root_dir /home/packerc/shared/albums/{SITE}/{SEASON}/ \
--log_dir /home/packerc/shared/season_captures/{SITE}/log_files/

In [ ]:
# check for duplicates - this can take a while for large batches >> 100k images
!python3 -m pre_processing.check_for_duplicates \
--root_dir /home/packerc/shared/albums/{SITE}/{SEASON}/ \
--log_dir /home/packerc/shared/season_captures/{SITE}/log_files/

In [ ]:
# Create Image Inventory
!python3 -m pre_processing.create_image_inventory \
--root_dir /home/packerc/shared/albums/{SITE}/{SEASON}/ \
--output_csv /home/packerc/shared/season_captures/{SITE}/inventory/{SEASON}_inventory_basic.csv \
--log_dir /home/packerc/shared/season_captures/{SITE}/log_files/

## Download Zooniverse Data

We can download Zooniverse data through Python.

In [ ]:
!python3 -m zooniverse_exports.get_zooniverse_export \
--password_file ~/keys/passwords.ini \
--project_id {PROJECT_ID} \
--output_file /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_classifications.csv \
--export_type classifications \
--log_dir /home/packerc/shared/zooniverse/Exports/{SITE}/

In [ ]:
# Get Zooniverse Subject Data
!python3 -m zooniverse_exports.get_zooniverse_export \
--password_file ~/keys/passwords.ini \
--project_id {PROJECT_ID} \
--output_file /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_subjects.csv \
--export_type subjects \
--log_dir /home/packerc/shared/zooniverse/Exports/{SITE}/

In [ ]:
# Extract Annotations
!python3 -m zooniverse_exports.extract_annotations \
--classification_csv /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_classifications.csv \
--output_csv /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_annotations.csv \
--workflow_id {WORKFLOW_ID} \
--workflow_version_min {WORKFLOW_VERSION_MIN}

In [ ]:
# Extract Subject Data
!python3 -m zooniverse_exports.extract_subjects \
--subject_csv /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_subjects.csv \
--output_csv /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_subjects_extracted.csv

## Post-Zooniverse Processing

The following codes are for downloading, extracting, and aggregating Zooniverse data and to create ConsensusReports.

In [ ]:
!python3 -m zooniverse_aggregations.aggregate_annotations_plurality \
--annotations /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_annotations.csv \
--output_csv /home/packerc/shared/zooniverse/Aggregations/{SITE}/{SEASON}_annotations_aggregated_plurality.csv \
--log_dir /home/packerc/shared/zooniverse/Aggregations/{SITE}/

In [ ]:
# Now we add subject infos to the aggregations
!python3 -m zooniverse_exports.merge_csvs \
--base_csv /home/packerc/shared/zooniverse/Aggregations/{SITE}/{SEASON}_annotations_aggregated_plurality.csv \
--to_add_csv /home/packerc/shared/zooniverse/Exports/{SITE}/{SEASON}_subjects_extracted.csv \
--output_csv /home/packerc/shared/zooniverse/Aggregations/{SITE}/{SEASON}_annotations_aggregated_plurality_info.csv \
--key subject_id

In [ ]:
tt = pd.read_csv('/home/packerc/shared/season_captures/ENO/captures/ENO_S1_action_list.csv')
tt.fillna('', inplace=True)

In [ ]:
tt.head

In [ ]:
df_show = qgrid.show_grid(tt, show_toolbar=True, )
df_show

In [ ]:
qgrid.show_grid(tt, row_edit_callback=can_edit_row)

In [ ]:
tt = tt.insert_(pd.Series())